In [ ]:
# PySpark in Microsoft Fabric Notebook

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count, desc
import os

# Create Spark session (Fabric usually provides this automatically)
spark = SparkSession.builder.appName("FabricPySparkAnalysis").getOrCreate()

# Path to CSV file in Fabric Lakehouse or local path
# In Fabric, replace with: "Files/sample_data.csv" or lakehouse path

# Use Fabric path if available, else local path
if os.path.exists("Files/sample_data.csv"):
    data_path = "Files/sample_data.csv"  # Fabric Lakehouse
    output_path = "Files/processed_results"
else:
    data_path = "data/sample_data.csv"   # Local
    output_path = "data/processed_results"

df = spark.read.option("header", True).option("inferSchema", True).csv(data_path)
try:
    # Load CSV with header and schema inference
    df = spark.read.option("header", True).option("inferSchema", True).csv(data_path)
except Exception as e:
    raise RuntimeError(f"Error loading data: {e}")

# Show first rows
print("📄 Raw Data Preview:")
df.show(5)

# Basic data cleaning: remove nulls in important columns
df_clean = df.dropna(subset=["Category", "Value"])

# Example aggregation: average value per category
agg_df = (
    df_clean.groupBy("Category")
    .agg(avg("Value").alias("AverageValue"), count("*").alias("Count"))
    .orderBy(desc("AverageValue"))
)

# Show results
print("Aggregated Results:")
agg_df.show()

# Save results back to Lakehouse (Parquet format)

agg_df.write.mode("overwrite").parquet(output_path)

print(f"✅ Processed data saved to: {output_path}")


📄 Raw Data Preview:
+--------+-----+
|Category|Value|
+--------+-----+
|       A|   10|
|       B|   15|
|       A|   20|
|       C|    5|
|       B|   25|
+--------+-----+

Aggregated Results:
+--------+------------+-----+
|Category|AverageValue|Count|
+--------+------------+-----+
|       B|        20.0|    2|
|       A|        15.0|    2|
|       C|         5.0|    1|
+--------+------------+-----+



Py4JJavaError: An error occurred while calling o96.parquet.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:314)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:1179)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:861)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:901)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:873)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:900)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs